In [ ]:
# Install required packages in the current Jupyter kernel
# Uncomment the following lines if you need to install these libraries
# If you run into permission issues, try with the --user option
#import sys
#!pip install -q rdflib networkx matplotlib
#!{sys.executable} -m pip install rdflib networkx matplotlib pandas stringdb --user


In [1]:
%load_ext autoreload
%autoreload 2
import sampler
import grabhogs_sparql
import map2string_fast
import addfrombloom
import rdflib
import SPARQLWrapper
import colour
import itertools
import networkx as nx
import glob
from matplotlib import pyplot as plt

In [2]:
datapath = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/402676.protein.links.rdf.v11.5.txt.gz'
# RDF graph loading
rg = sampler.load_graph(datapath)

datapath2 = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.info.rdf.v11.5/402676.protein.info.rdf.v11.5.txt.gz'
# RDF graph loading
rg_info  = sampler.load_graph(datapath2)

rdflib Graph loaded successfully with 2620310 triples
rdflib Graph loaded successfully with 14559 triples


In [3]:
links = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/*.protein.links.rdf.v11.5.txt.gz'
linkfiles = glob.glob(links)
linkfiles = { l:{ 'links':l , 'info':l.replace('protein.links' , 'protein.info' ) } for l in linkfiles}
print(len(linkfiles ))

1831


In [4]:
subjs = rg.subjects( unique = True)
seed = next(subjs)
print(seed)

https://string-db.org/network/402676.B6K0F0


In [5]:
subg = sampler.sample( rg = rg , seed = seed,  layer_limit= 2 , sample_run = 20 )
print(set([p for p in subg.predicates()]))
print("rdflib Graph sampled successfully with {} triples".format(len(subg)))

176 656
{rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/any-confidence'), rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff')}
rdflib Graph sampled successfully with 551 triples


In [6]:
subg = sampler.add_xrefs( rg_info , subg )
print("rdflib Graph annotated successfully with {} triples".format(len(subg)))
print(set([p for p in subg.predicates()]))
cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot")
i = 0 
for s,p,o in subg.triples((None, cross_ref, None)):
    print(s,p,o)
    i+= 1
    if i > 10:
        break

83
83
rdflib Graph annotated successfully with 634 triples
{rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff'), rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot'), rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/any-confidence')}
https://string-db.org/network/402676.B6JVH1 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JVH1
https://string-db.org/network/402676.B6JWV4 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JWV4
https://string-db.org/network/402676.B6JVX3 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JVX3
https://string-db.org/network/402676.B6JWG8 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JWG8
https://string-db.org/network/402676.B6JW85 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JW85
https://string-db.o

In [7]:
#proteins_by_species , results_subj_para , results_subj_ortho  = grabhogs_sparql.grab_hogs( subg , cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot") )

In [8]:
#print(results_subj_ortho)

In [9]:
orthograph =  grabhogs_sparql.grab_hogs_graph( subg , cross_ref , sparql_endpoint= None
            , USE_CASE = 1 , verbose = True , cross_ref_prop = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot"))

83
Total time for 166 SPARQL queries: 80.63025951385498 seconds (multiple batch calls in: 0 cases)
Num errors: 0


In [10]:
from rdflib import Graph, URIRef
print("rdflib Graph annotated successfully with {} triples".format(len(orthograph)))
#get all species
taxa = [ 'protein1_uniprot_taxon_orth' , 'protein1_uniprot_taxon_para']
species = set( [ o for t in taxa  for s,p,o in orthograph.triples((None, URIRef(t), None))  ])


rdflib Graph annotated successfully with 1324517 triples


In [13]:
#get all the results for that species
taxa = [ 'orth_protein_uniprot_taxon_orth' , 'para_protein_uniprot_taxon_para']
#get all proteins for each species
prots_by_species = { spec: set([s  for s,p,o in orthograph.triples((None, URIRef(t), spec ))]) for t in taxa for spec in species  }
prots_by_species = { spec:prots_by_species[spec] for spec in prots_by_species if len(prots_by_species[spec])   }
prots_by_species = { spec:[ p.replace('http://purl.uniprot.org/uniprot/' , '' ) for p in prots_by_species[spec] ] for spec in prots_by_species }


In [ ]:
server = 'dna076'
ortho_xrefgraph = map2string_fast.mapall(prots_by_species , serverurl= "http://"+server+":3030/string_fuseki/sparql" , retgraph = True)

In [ ]:
orthograph += ortho_xrefgraph
subg += orthograph
#we have interactions for one species and ortho info to all others

In [ ]:
filters = addfrombloom.load_filters()

In [ ]:
#lets add the interactions for all using the bloom filters
print( len(orthograph ) , set([p for p in orthograph.predicates() ]) )
#get string ids by species
pred = rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot')
interactions = []
for spec in prots_by_species:
    stringids = [ s for prot in prots_by_species[spec] for s,p,o in orthograph.triples((None, pred , URIRef('http://purl.uniprot.org/uniprot/'+prot) )) ]
    stringids = [ s.replace('https://string-db.org/network/' , '' ) for s in stringids ]
    
    if len(stringids ) > 2 :
        interactions += addfrombloom.check_allvall( objects = stringids , urlstring = 'https://string-db.org/network/' , filters = filters )
[subg.add(t) for t in interactions]

In [ ]:
#halelujah we have a graph with everything in it
#serialize to turtle format
v = subg.serialize(format="ttl")
with open('testgraph.ttl', 'w') as graphout:
    graphout.write(v)

In [ ]:
print(len(subg))
readg = Graph()
readg.parse('testgraph.ttl')
print(len(readg))
#we can save the subgraphs in rdf...


In [ ]:
# create a new heterodata object
from torch_geometric.data import HeteroData , Data
import torch_geometric.transforms as T
import torch_geometric.utils 
from rdflib import RDF
import numpy as np
import scipy.sparse
import torch


def sparse2pairs(sparsemat, matrows = None):
    '''
    This functino takes a sparse matrix and returns a list of pairs of the non zero entries
    args:
        sparsemat: a sparse matrix
        matrows: a list of the matrix rows to keep
    Returns:    
        a list of pairs of the non zero entries
    '''
    if matrows :
        sparsemat = sparsemat[matrows,:]
        sparsemat = sparsemat[:,matrows]
    sparsemat = scipy.sparse.find(sparsemat)
    return np.vstack([sparsemat[0],sparsemat[1]])

def test_heterograph(rdf_graph , verbose = True):
    data = HeteroData()
    # assign edge types from the predicate
    edge_types = set([p for s,p,o in rdf_graph])
    edge_types = {e:i for i,e in enumerate(edge_types)}
    uris = {}
    subtypes = {}
    for sub, pred, obj in rdf_graph:
        for o in (sub,obj):
            if isinstance(o, URIRef):
                uri_type = rdf_graph.qname(o).split(":")[0]  # get the namespace prefix
                if uri_type not in uris:
                    uris[uri_type] = []
                    subtypes[uri_type] = set([])
                uris[uri_type].append(o)
                subtypes[uri_type].add( ''.join(o.n3().split('/')[0:-1] )  )
    if verbose == True:
        for t in subtypes:
            #compile the x data matrix for each subtype
            subtype_dict = { ty:i for i,ty in enumerate(subtypes[t])}
            indices = [ subtype_dict[''.join(o.n3().split('/')[0:-1])] for o in uris[t] ]
            x= np.zeros( (len(uris[t]), len(subtype_dict)))
            x[:,indices] = 1
            #for now this is 1hot for subtype
            #add some more descriptive data here if you have it
            data[t].x = torch.tensor(x , dtype=torch.float )
    node_index_by_type = { uritype : { n:i for i,n in enumerate( set(uris[uritype]) ) } for uritype in uris }
    #todo add the diff types of uris within each namespace as 1hot encoded for x attribute of each node
    for t1,t2 in itertools.product(node_index_by_type,node_index_by_type):
            rows = node_index_by_type[t1]
            columns = node_index_by_type[t2]
            if verbose == True:
                print(t1,len(rows),t2, len(columns)) 
                
            for edge_type in edge_types:
                
                # create a dictionary of nodes
                
                triples =  [ (s,p,o) for s,p,o in rdf_graph.triples((None, edge_type, None))  if ( s in rows and o in columns )  ]
                
                if len(triples)>0:
                    adj = scipy.sparse.lil_matrix((len(rows), len(columns)))
                    for s,p,o in triples:    
                        adj[rows[s], columns[o]] = 1
                    if verbose == True:
                        print(adj.sum())
                        pass
                    #between subgraphs
                    data[ t1 , p.n3() , t2 ].edge_index = torch.tensor( sparse2pairs(adj) ,  dtype=torch.long )
                    if t1 == t2:
                        #within a subgraph of the same namespace
                        data[ t1 , p.n3() , t2 ].edge_index  = torch_geometric.utils.add_self_loops(data[ t1 , p.n3() , t2 ].edge_index )
    data = T.ToUndirected()(data)
    return data
data = test_heterograph(subg)
print(data)

In [ ]:
#get a model


In [ ]:
#sample edges


In [ ]:
#train


In [ ]:
#pred 